In [19]:
# Import packages
import torch
torch.cuda.empty_cache()
from diffusers import DiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionDepth2ImgPipeline, UniPCMultistepScheduler
from PIL import Image
import os
import cv2
from torchvision.io.video import read_video, write_video
import torchvision.transforms.functional as F
import numpy as np

In [6]:
# Create the pipeline
#pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", use_safetensors=True)

def SDPipeline(input_dir, output_dir, prompt, n_prompt):

    device = "cuda"

    # Different types of pipelines: img2img and depth2img
    
    pipe = StableDiffusionDepth2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-2-depth",torch_dtype=torch.float16, cache_dir="/work3/s204134/HF_cache")
    pipe.safety_checker = None
    
    pipe = pipe.to(device)


    image_dir = os.path.join(".", f"{input_dir}")
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]


    # Input the name of the folder where you want the data to be stored
    #relativePath = f"./baselineStableDiffusion/{dirname}/"
    #savePath =  os.path.abspath(relativePath)

    #savePath = f"./baselineStableDiffusion/{dirname}/"
    savePath = os.path.join(".", output_dir)


    try:

        # creating a folder named data
        if not os.path.exists(savePath):
            #print(os.path.exists(savePath))
            os.makedirs(output_dir, exist_ok=True)
            print(os.path.exists(savePath))


    # if not created then raise error
    except OSError:
        print ('Error: Creating directory of data')
        print(os.path.exists(savePath))




    for idx, image_file in enumerate(image_files):
        image_path = os.path.join(image_dir, image_file)
        init_image = Image.open(image_path)
        init_image.resize((896, 512))

        output_filename = f"reimaginedframe{idx}.png"
        output_path = os.path.join(savePath, output_filename)
        
        
        image = pipe(prompt=prompt, image=init_image, negative_prompt=n_prompt, strength=0.7)[0]
        image[0].save(output_path)
            

In [7]:
############################# Pipeline #########################
input_dir = 

out_dir = f"{namevid}Retouched"

prompt = list_of_prompts[idx]
n_prompt = "bad anatomy, ugly, blurry, deformed, bad, cut-off, spotty, overlapping"

SDPipeline(frame_out, out_dir, prompt, n_prompt)


NameError: name 'namevid' is not defined

In [10]:
pipe = StableDiffusionDepth2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-depth", safety_checker=None, torch_dtype=torch.float16,
    # cache_dir="/work3/s204158/HF_cache"
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_model_cpu_offload()

text_encoder\model.safetensors not found
Keyword arguments {'safety_checker': None} are not expected by StableDiffusionDepth2ImgPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [21]:
input_dir = "movieClips/BSVid.mp4"
output_dir = "BSVid_baseline.mp4"
prompt = "A man in van gogh style"

input_video, _, info = read_video(input_dir, pts_unit="sec", output_format="TCHW")

In [17]:
output_video = []

for i in range(0, len(input_video)):
  curr = input_video[i]
  curr = F.to_pil_image(curr)
  curr = curr.resize((512, 512))


  
  outputImg = pipe(prompt, curr, num_inference_steps=20).images[0]
  output_video.append(outputImg)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
output_video_pytorch = []

for frame in output_video:
  img = frame
  img_array = np.array(img)
  tensor = torch.from_numpy(img_array)
  output_video_pytorch.append(tensor)

output_video_pytorch = torch.stack(output_video_pytorch)
write_video(output_dir, output_video_pytorch, fps=info["video_fps"])    